In [1]:
# Initial imports
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
from datetime import datetime
import numpy as np
#import tensorflow as tf

#  Import and read the file.
supply_df = pd.read_csv("CAISO-ALL-SUPPLY-DATA.csv")
supply_df.head()


,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time,DateTime
0,50036.0,5568.0,78.0,2353.0,-447.0,2265.0,21.0,0.0,7/5/21,11:50:00,7/5/21 11:50
1,8695.0,26540.0,5081.0,4575.0,119.0,2248.0,21.0,0.0,8/18/20,16:25:00,8/18/20 16:25
2,9301.0,26159.0,4408.0,5035.0,67.0,2250.0,23.0,0.0,8/18/20,15:55:00,8/18/20 15:55
3,9343.0,26132.0,4382.0,5032.0,49.0,2250.0,22.0,0.0,8/18/20,15:50:00,8/18/20 15:50
4,9283.0,26015.0,4300.0,5278.0,63.0,2248.0,22.0,0.0,8/18/20,15:40:00,8/18/20 15:40


In [2]:
supply_df['Sum'] = supply_df['Renewables'] + supply_df['Large hydro'] + supply_df['Imports'] + supply_df['Batteries'] + supply_df['Nuclear'] + supply_df['Coal'] + supply_df['Other'] + supply_df['Natural gas']


In [3]:
baby_df = supply_df['Time'].str.split(":", expand=True)
print(baby_df)

         0   1   2
0       11  50  00
1       16  25  00
2       15  55  00
3       15  50  00
4       15  40  00
...     ..  ..  ..
412947   2  35  00
412948   2  40  00
412949   2  45  00
412950   2  50  00
412951   2  55  00

[412952 rows x 3 columns]


In [4]:
#Format the baby_df into int.
baby_df[0] = baby_df[0].astype(int)
baby_df[1] = baby_df[1].astype(int)
#Clean baby_df
baby_df= baby_df.drop([2],1)

# Rename columns and create years value for calculating demand_EV
baby_df = baby_df.rename(columns={baby_df.columns[0]: 'Hours'})
baby_df = baby_df.rename(columns={baby_df.columns[1]: 'Minutes'})
baby_df

C:\Users\the_d\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """


,Hours,Minutes
0,11,50
1,16,25
2,15,55
3,15,50
4,15,40
...,...,...
412947,2,35
412948,2,40
412949,2,45
412950,2,50


In [5]:
supply_df['Date'] = pd.to_datetime(supply_df['Date'])

#Create a new column for'Year'
baby_df['Year'] = supply_df['Date'].dt.year
#Convert to integers to be used in computations
baby_df['Year'] = baby_df['Year'].astype(int)

baby_df.head(100)

,Hours,Minutes,Year
0,11,50,2021
1,16,25,2020
2,15,55,2020
3,15,50,2020
4,15,40,2020
...,...,...,...
95,17,15,2020
96,17,15,2020
97,15,50,2020
98,15,50,2020


In [6]:
baby_df['DemandMW'] = 0.0

mask = (baby_df['Year'] == 2018) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)
mask1 = (baby_df['Year'] == 2019) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)
mask2 = (baby_df['Year'] == 2020) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)


baby_df.loc[mask,'DemandMW'] = 6.2356
baby_df.loc[mask1,'DemandMW'] = 8.2818
baby_df.loc[mask2,'DemandMW'] = 9.9174

In [7]:
#Make a date string column 
date_df = pd.DataFrame()
supply_df['Date1']=supply_df['Date'].astype(str)

date_df = supply_df['Date1'].str.split("-", expand=True)
date_df['DateAttribute'] = date_df[1] + date_df[2]
date_df = date_df.drop([0,1,2],1)


C:\Users\the_d\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  import sys


In [8]:
calc_df = pd.merge(baby_df, date_df, left_index=True, right_index=True, how='outer')
calc_df.head(-5)

,Hours,Minutes,Year,DemandMW,DateAttribute
0,11,50,2021,0.0,0705
1,16,25,2020,0.0,0818
2,15,55,2020,0.0,0818
3,15,50,2020,0.0,0818
4,15,40,2020,0.0,0818
...,...,...,...,...,...
412942,2,10,2022,0.0,0314
412943,2,15,2022,0.0,0314
412944,2,20,2022,0.0,0314
412945,2,25,2022,0.0,0314


In [9]:
def fun(datetimething):
     return datetimething.timetuple().tm_yday

In [10]:
supply_df['Date'] = supply_df['Date'].apply(fun)
supply_df.head(1300)

,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time,DateTime,Sum,Date1
0,50036.0,5568.0,78.0,2353.0,-447.0,2265.0,21.0,0.0,186,11:50:00,7/5/21 11:50,59874.0,2021-07-05
1,8695.0,26540.0,5081.0,4575.0,119.0,2248.0,21.0,0.0,231,16:25:00,8/18/20 16:25,47279.0,2020-08-18
2,9301.0,26159.0,4408.0,5035.0,67.0,2250.0,23.0,0.0,231,15:55:00,8/18/20 15:55,47243.0,2020-08-18
3,9343.0,26132.0,4382.0,5032.0,49.0,2250.0,22.0,0.0,231,15:50:00,8/18/20 15:50,47210.0,2020-08-18
4,9283.0,26015.0,4300.0,5278.0,63.0,2248.0,22.0,0.0,231,15:40:00,8/18/20 15:40,47209.0,2020-08-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1295,13006.0,20240.0,3017.0,4564.0,-15.0,2257.0,20.0,0.0,232,12:55:00,8/19/20 12:55,43089.0,2020-08-19
1296,13006.0,20240.0,3017.0,4564.0,-15.0,2257.0,20.0,0.0,233,12:55:00,8/20/20 12:55,43089.0,2020-08-20
1297,5842.0,24279.0,4693.0,5863.0,144.0,2244.0,22.0,0.0,232,19:15:00,8/19/20 19:15,43087.0,2020-08-19
1298,5842.0,24279.0,4693.0,5863.0,144.0,2244.0,22.0,0.0,233,19:15:00,8/20/20 19:15,43087.0,2020-08-20


In [11]:
supply_df = supply_df.drop(['Date1'],1)
final_df = pd.merge(supply_df, calc_df, left_index=True, right_index=True, how='outer')
final_df
final_df['DateTime'] = pd.to_datetime(final_df['DateTime'])
#final_df['AvgSUM(3yr)'] = 0.0
#final_df['AvgSUM(3yr)'] = final_df['AvgSUM(3yr)'].astype(float)


C:\Users\the_d\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [12]:
df1= pd.DataFrame()
df1 = final_df.groupby(['DateAttribute','Time'])['Sum'].agg('mean')
df1 = df1.reset_index()
df1 = df1.rename(columns={df1.columns[2]: 'AvgSUM(3yr)'}).round(2)
df2 = df1.copy

print(len(df1))
df1.dtypes


105408


DateAttribute     object
Time              object
AvgSUM(3yr)      float64
dtype: object

In [13]:

df1['Date1']= "2020" + df1['DateAttribute']
df1['pDateTime'] = df1['Date1'] +" " +df1['Time']
df1['Date2']= pd.to_datetime(df1['pDateTime'])
df1=df1.set_index('Date2')       
                                            
df1=df1.sort_index(ascending=True)
df1=df1.reset_index()
ev_df2=df1.drop(['DateAttribute','Date1','pDateTime','Time'],1)

C:\Users\the_d\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [14]:
df1 = df1.drop(df1.index[16992:17280])
df1 = df1.reset_index(drop=True)
print(len(df1))
df1

105120


,Date2,DateAttribute,Time,AvgSUM(3yr),Date1,pDateTime
0,2020-01-01 00:00:00,0101,0:00:00,23008.50,20200101,20200101 0:00:00
1,2020-01-01 00:05:00,0101,0:05:00,22992.00,20200101,20200101 0:05:00
2,2020-01-01 00:10:00,0101,0:10:00,22944.25,20200101,20200101 0:10:00
3,2020-01-01 00:15:00,0101,0:15:00,22891.00,20200101,20200101 0:15:00
4,2020-01-01 00:20:00,0101,0:20:00,22834.25,20200101,20200101 0:20:00
...,...,...,...,...,...,...
105115,2020-12-31 23:35:00,1231,23:35:00,23266.50,20201231,20201231 23:35:00
105116,2020-12-31 23:40:00,1231,23:40:00,23171.50,20201231,20201231 23:40:00
105117,2020-12-31 23:45:00,1231,23:45:00,23105.00,20201231,20201231 23:45:00
105118,2020-12-31 23:50:00,1231,23:50:00,23031.75,20201231,20201231 23:50:00


In [15]:
df1=df1.drop(['Date2','Date1','pDateTime'],1)

df1['Date2']= "2019" + df1['DateAttribute']
df1['psDateTime'] = df1['Date2'] +" " +df1['Time']
df1['Date2']= pd.to_datetime(df1['psDateTime'])
df1=df1.set_index('Date2')       
                                            
df1=df1.sort_index(ascending=True)
df1=df1.reset_index()
ev_df1=df1.drop(['DateAttribute','psDateTime','Time'],1)
ev_df1

C:\Users\the_d\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
C:\Users\the_d\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # Remove the CWD from sys.path while we load stuff.


,Date2,AvgSUM(3yr)
0,2019-01-01 00:00:00,23008.50
1,2019-01-01 00:05:00,22992.00
2,2019-01-01 00:10:00,22944.25
3,2019-01-01 00:15:00,22891.00
4,2019-01-01 00:20:00,22834.25
...,...,...
105115,2019-12-31 23:35:00,23266.50
105116,2019-12-31 23:40:00,23171.50
105117,2019-12-31 23:45:00,23105.00
105118,2019-12-31 23:50:00,23031.75


In [16]:
df1=df1.drop(['Date2','psDateTime'],1)

df1['Date2']= "2018" + df1['DateAttribute']
df1['psDateTime'] = df1['Date2'] +" " +df1['Time']
df1['Date2']= pd.to_datetime(df1['psDateTime'])
df1=df1.set_index('Date2')       
                                            
df1=df1.sort_index(ascending=True)
df1=df1.reset_index()
ev_df=df1.drop(['DateAttribute','psDateTime','Time'],1)
ev_df

C:\Users\the_d\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.
C:\Users\the_d\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  # Remove the CWD from sys.path while we load stuff.


,Date2,AvgSUM(3yr)
0,2018-01-01 00:00:00,23008.50
1,2018-01-01 00:05:00,22992.00
2,2018-01-01 00:10:00,22944.25
3,2018-01-01 00:15:00,22891.00
4,2018-01-01 00:20:00,22834.25
...,...,...
105115,2018-12-31 23:35:00,23266.50
105116,2018-12-31 23:40:00,23171.50
105117,2018-12-31 23:45:00,23105.00
105118,2018-12-31 23:50:00,23031.75


In [17]:
ev_avg = ev_df.append(ev_df1)
ev_avg1 = ev_avg.append(ev_df2)
ev_avg1=ev_avg1.set_index('Date2')       
                                            
ev_avg1=ev_avg1.sort_index(ascending=True)
ev_avg1=ev_avg1.reset_index()


print(len(ev_avg1))
ev_avg1

315648


,Date2,AvgSUM(3yr)
0,2018-01-01 00:00:00,23008.50
1,2018-01-01 00:05:00,22992.00
2,2018-01-01 00:10:00,22944.25
3,2018-01-01 00:15:00,22891.00
4,2018-01-01 00:20:00,22834.25
...,...,...
315643,2020-12-31 23:35:00,23266.50
315644,2020-12-31 23:40:00,23171.50
315645,2020-12-31 23:45:00,23105.00
315646,2020-12-31 23:50:00,23031.75


In [18]:
y = ev_avg1.loc[(ev_avg1['Date2'] == '2018-04-10 00:00:00')]
print(y.index)

Int64Index([28512], dtype='int64')


In [19]:
ev_avg1 = ev_avg1.drop(ev_avg.index[0:28512])
ev_avg1 = ev_avg1.reset_index(drop=True)
ev_avg1 = ev_avg1.rename(columns={ev_avg1.columns[0]: 'DateTime'})
ev_avg1['AvgSUM(3yr)'] = ev_avg1['AvgSUM(3yr)'].astype(float)
print(len(ev_avg))

ev_avg1


210240


,DateTime,AvgSUM(3yr)
0,2018-04-10 00:00:00,21718.50
1,2018-04-10 00:05:00,21665.50
2,2018-04-10 00:10:00,21622.00
3,2018-04-10 00:15:00,21495.50
4,2018-04-10 00:20:00,21408.50
...,...,...
287131,2020-12-31 23:35:00,23266.50
287132,2020-12-31 23:40:00,23171.50
287133,2020-12-31 23:45:00,23105.00
287134,2020-12-31 23:50:00,23031.75


In [21]:
from sqlalchemy import create_engine

# SQLAlchemy connectable
cnx = create_engine('sqlite:///db-carenewable.db').connect()

In [22]:
# Write supply_df to SQLAlchemy table
ev_avg1.to_sql('AVERAGE-DEMAND-2020', cnx)

In [23]:
# Close SQLAlchemy connection
cnx.close()

In [20]:
ev_avg1=ev_avg1.set_index('DateTime')
final_df = final_df.set_index('DateTime')


In [21]:
#tried a dictionary - got a bunch of NaN
sum_dict = ev_avg1.to_dict()
#print(sum_dict)

#final_df['AvgSUM(3yr)'] = ev_avg1[ev_avg1['DateTime'].isin(final_df['DateTime'])]['AvgSUM(3yr)'].values

In [22]:
final_df['AvgSUM(3yr)'] = pd.Series(sum_dict)

final_df


,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time,Unix Timestamp,Sum,Hours,Minutes,Year,DemandMW,DateAttribute,AvgSUM(3yr)
DateTime,,,,,,,,,,,,,,,,,,
2018-04-10 00:00:00,2005.0,8736.0,3428.0,7108.0,-8.0,1616.0,-6.0,0.0,100,0:00:00,1.523318e+09,22879.0,0,0,2018,6.2356,0410,NaN
2018-04-10 00:05:00,2005.0,8722.0,3425.0,7029.0,1.0,1618.0,-6.0,0.0,100,0:05:00,1.523319e+09,22794.0,0,5,2018,6.2356,0410,NaN
2018-04-10 00:10:00,2048.0,8739.0,3464.0,6873.0,6.0,1619.0,-6.0,0.0,100,0:10:00,1.523319e+09,22743.0,0,10,2018,6.2356,0410,NaN
2018-04-10 00:15:00,1979.0,8696.0,3470.0,6744.0,-14.0,1619.0,-6.0,0.0,100,0:15:00,1.523319e+09,22488.0,0,15,2018,6.2356,0410,NaN
2018-04-10 00:20:00,1980.0,8605.0,3454.0,6757.0,-38.0,1616.0,-6.0,0.0,100,0:20:00,1.523320e+09,22368.0,0,20,2018,6.2356,0410,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 23:35:00,3183.0,8540.0,764.0,8800.0,122.0,1144.0,12.0,0.0,366,23:35:00,1.609458e+09,22565.0,23,35,2020,9.9174,1231,NaN
2020-12-31 23:40:00,3294.0,8450.0,718.0,8804.0,99.0,1144.0,12.0,0.0,366,23:40:00,1.609458e+09,22521.0,23,40,2020,9.9174,1231,NaN
2020-12-31 23:45:00,3410.0,8300.0,731.0,8725.0,90.0,1143.0,13.0,0.0,366,23:45:00,1.609458e+09,22412.0,23,45,2020,9.9174,1231,NaN
